## Exchange Rates and Food Prices

### Introduction

There are two datasets for this project. A dataset of exchange rates was sourced from Central Bank of Nigeria via https://www.cbn.gov.ng/rates/exrate.asp Another dataset of food prices in Nigeria was sourced from the Nigeria Bureau of Statistics via https://africaopendata.org/dataset/food-prices-for-nigeria.

The dataset covers exchange rates of countries from 2004-2021. Our interest is in the Nigeria US Dollar exchange rates from 2017 - 2021.  The food prices datasets contains prices of some selected food items from 2017 - 2021.

#### Objective
Our objective here is to answer the following questions.

*   What is the trend of the USD/NGN exchange rate between 2017 and 2019?
*   During the covid pandemic period (2020), how does the exchange rate fair?
*   What is the correlation between the exchange rates and food prices?
*   How does this change in (iii) above compare with 2020-2021?
*   Which food items are most correlated with the exchange rate fluctuations (if any)?
*   What is the rate of increase in food prices with change in exchange rates
*   What period has the highest increment in food prices?

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
# Load the datasets
monthly_exc_rates = pd.read_csv('./datasets/MonthlyAverageExhRateNG.csv')

In [3]:
foodprices = pd.read_csv('./datasets/food_prices_ng.csv')

## Data Cleaning

##### Food prices dataset cleaning

In [4]:
# inspect first few rows
foodprices.head()

,Item Labels,Jan-17,Feb-17,Mar-17,Apr-17,May-17,Jun-17,Jul-17,Aug-17,Sep-17,...,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Lowest,Highest,Month-on-Month,Year-on-Year
0,Agric eggs medium size,512.99,520.16,507.69,518.66,522.71,514.66,498.50,485.19,487.51,...,556.47,577.55,585.64,604.99,614.89,629.82,Borno (411.67),Bayelsa (800),2.43,27.31
1,Agric eggs(medium size price of one),47.42,42.90,43.93,46.22,45.73,45.28,44.32,42.92,45.72,...,51.22,52.73,53.20,54.62,55.49,57.35,Taraba (41.51),Ogun (70.06),3.35,28.15
2,"Beans brown,sold loose",353.60,337.11,353.28,357.19,365.86,374.26,382.35,370.25,404.84,...,471.24,485.44,496.03,492.13,478.76,490.19,Bauchi (234.29),Anambra (899.4),2.39,46.47
3,Beans:white black eye. sold loose,305.53,309.85,318.45,324.04,332.28,339.64,343.96,335.71,358.13,...,431.79,444.21,457.98,461.59,464.74,476.76,Bauchi (235.22),Ebonyi (887.77),2.59,54.61
4,Beef Bone in,1001.24,995.65,1010.29,1035.45,1123.65,1129.02,1128.94,1151.38,1078.49,...,1220.33,1262.10,1274.20,1320.73,1339.77,1366.24,Taraba (975.2),Abuja (1885.99),1.98,26.37


In [5]:
# check the data type of the columns
foodprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 64 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Item Labels     44 non-null     object 
 1   Jan-17          43 non-null     float64
 2   Feb-17          43 non-null     float64
 3   Mar-17          43 non-null     float64
 4   Apr-17          43 non-null     float64
 5   May-17          43 non-null     float64
 6   Jun-17          43 non-null     float64
 7   Jul-17          43 non-null     float64
 8   Aug-17          43 non-null     float64
 9   Sep-17          43 non-null     float64
 10  Oct-17          43 non-null     float64
 11  Nov-17          43 non-null     float64
 12  Dec-17          43 non-null     float64
 13  Jan-18          43 non-null     float64
 14  Feb-18          43 non-null     float64
 15  Mar-18          43 non-null     float64
 16  Apr-18          43 non-null     float64
 17  May-18          43 non-null     float

From the initial inspection of the food prices datasets, we will remove some columns that are not so relevant to our goal `Lowest`, `Highest`, `Month-on-Month`, and `Year-on-Year`. We'll also Transpose the dataset for each of the food items into respective columns, with each row being an instance of the prices for a particular month.

In [7]:
# Remove columns
foodprices.drop(['Lowest', 'Highest',
                 'Month-on-Month','Year-on-Year'],
               axis=1, inplace=True)
# Transpose the dataset
foodprices = foodprices.T
foodprices.head(3)


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
Item Labels,Agric eggs medium size,Agric eggs(medium size price of one),"Beans brown,sold loose",Beans:white black eye. sold loose,Beef Bone in,"Beef,boneless",Bread sliced 500g,Bread unsliced 500g,Broken Rice (Ofada),Catfish (obokun) fresh,...,"Rice,imported high quality sold loose",Sweet potato,Tilapia fish (epiya) fresh,Titus:frozen,Tomato,"Vegetable oil:1 bottle,specify bottle",Wheat flour: prepacked (golden penny 2kg),Yam tuber,NaN,Source: The Nigeria Bureau of Statistics
Jan-17,512.99,47.42,353.6,305.53,1001.24,1249.48,302.87,270.25,377.41,899.58,...,402.01,129.43,792.18,884.82,247.55,495.29,626.53,210.58,NaN,NaN
Feb-17,520.16,42.9,337.11,309.85,995.65,1270.67,299.75,264.86,392.04,885.13,...,410.58,127.23,795.27,942.23,236.62,507.36,621.39,215.55,NaN,NaN


In [8]:
# Remove the last two redundant columns 
foodprices.drop([43,44], axis=1, inplace=True)

Let us replace the column names with the first row and then remove that row. We will keep the index for easy merging of the second dataset.

In [23]:
# Retrieve the column names from the first row
col_names = np.array(foodprices.loc['Item Labels'])
# rename the columns appropriately
foodprices.columns = col_names
# remove the redundant first row
foodprices.drop(['Item Labels'], axis=0, inplace=True)


In [25]:
# Reindex the data and add new column period
foodprices['period'] = foodprices.index
foodprices.index = pd.Index(np.arange(len(foodprices)))

In [26]:
foodprices.tail(3)

,Agric eggs medium size,Agric eggs(medium size price of one),"Beans brown,sold loose",Beans:white black eye. sold loose,Beef Bone in,"Beef,boneless",Bread sliced 500g,Bread unsliced 500g,Broken Rice (Ofada),Catfish (obokun) fresh,...,Rice Medium Grained,"Rice,imported high quality sold loose",Sweet potato,Tilapia fish (epiya) fresh,Titus:frozen,Tomato,"Vegetable oil:1 bottle,specify bottle",Wheat flour: prepacked (golden penny 2kg),Yam tuber,period
56,604.99,54.62,492.13,461.59,1320.73,1768.14,384.73,351.95,473.75,1232.25,...,454.28,546.77,192.07,1067.47,1426.42,342.25,812.94,911.88,306.87,Sep-21
57,614.89,55.49,478.76,464.74,1339.77,1768.78,382.77,359.6,475.54,1242.54,...,463.38,553.23,192.14,1074.97,1424.58,347.47,825.46,912.72,313.05,Oct-21
58,629.82,57.35,490.19,476.76,1366.24,1812.03,404.2,366.46,480.65,1276.58,...,476.67,568.75,202.25,1101.38,1451.35,349.6,876.47,950.6,327.53,Nov-21


In [49]:
foodprices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 44 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Agric eggs medium size                     59 non-null     object
 1   Agric eggs(medium size price of one)       59 non-null     object
 2   Beans brown,sold loose                     59 non-null     object
 3   Beans:white black eye. sold loose          59 non-null     object
 4   Beef Bone in                               59 non-null     object
 5   Beef,boneless                              59 non-null     object
 6   Bread sliced 500g                          59 non-null     object
 7   Bread unsliced 500g                        59 non-null     object
 8   Broken Rice (Ofada)                        59 non-null     object
 9   Catfish (obokun) fresh                     59 non-null     object
 10  Catfish :dried                          

In [50]:
# change the column data type to float except period
for col in foodprices.columns:
    if col != 'period':
        foodprices[col] = foodprices[col].astype(float)
        

In [51]:
foodprices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 44 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Agric eggs medium size                     59 non-null     float64
 1   Agric eggs(medium size price of one)       59 non-null     float64
 2   Beans brown,sold loose                     59 non-null     float64
 3   Beans:white black eye. sold loose          59 non-null     float64
 4   Beef Bone in                               59 non-null     float64
 5   Beef,boneless                              59 non-null     float64
 6   Bread sliced 500g                          59 non-null     float64
 7   Bread unsliced 500g                        59 non-null     float64
 8   Broken Rice (Ofada)                        59 non-null     float64
 9   Catfish (obokun) fresh                     59 non-null     float64
 10  Catfish :dried              

#### Exchange rates dataset cleaning

In [27]:
# initial inspection
monthly_exc_rates.head()

,Year,Month,DAS (USD),IFEM (USD),BDC (USD),GB Pounds,EURO,CFA Franc,Unnamed: 8
0,2004,1,136.08,137.76,147.65,247.93,171.67,0.16,NaN
1,2004,2,135.16,136.44,142.95,252.78,171.09,0.20,NaN
2,2004,3,134.47,134.80,139.92,245.47,164.88,0.25,NaN
3,2004,4,133.51,137.23,138.85,240.26,159.87,0.24,NaN
4,2004,5,133.01,134.81,139.64,237.80,159.75,0.24,NaN


In [28]:
monthly_exc_rates.Year.unique()

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021])

Let us retain only the columns we need. The columns are `Year`, `Month` and `IFEM (USD)`. We will create another column `period` to hold the Month and year to match the index format ('Jan-17') of the food prices dataset.
We are interested in data from 2017 - 2021.

In [29]:
# Extract data for the last five years
exchange_rates = monthly_exc_rates[monthly_exc_rates['Year'] >= 2017]

In [30]:
exchange_rates = exchange_rates[['Year','Month','IFEM (USD)']]

In [31]:
exchange_rates.head()

,Year,Month,IFEM (USD)
156,2017,1,305.20
157,2017,2,305.31
158,2017,3,306.40
159,2017,4,306.05
160,2017,5,305.54


In [32]:
# Rename `IFEM (USD)`
exchange_rates.rename(columns={'IFEM (USD)': 'Usd rate'}, inplace=True)

In [33]:
exchange_rates.tail()

,Year,Month,Usd rate
203,2020,12,381.0
204,2021,1,381.0
205,2021,2,381.0
206,2021,3,381.0
207,2021,4,381.0


Let's create a new column with format that matches the index of the food prices dataset. We will also create a date column.

In [37]:
# create the format 'Jan-17' and datetime columns 
month_code = {'1':'Jan','2':'Feb','3':'Mar','4':'Apr','5':'May','6':'Jun',
             '7':'Jul','8':'Aug','9':'Sep','10':'Oct','11':'Nov','12':'Dec'}
month_year = []
date = []
for m, y in zip(exchange_rates['Month'], exchange_rates['Year']):
    dt = month_code[str(m)] + '-' + str(y)[-2:]
    date_fmt = "28-"+str(m)+"-"+ str(y)
    month_year.append(dt)
    date.append(datetime.strptime(date_fmt, "%d-%m-%Y"))
exchange_rates['period'] = month_year
exchange_rates['Year'] = date


In [38]:
exchange_rates.head(3)

,Year,Month,Usd rate,period
156,2017-01-28,1,305.20,Jan-17
157,2017-02-28,2,305.31,Feb-17
158,2017-03-28,3,306.40,Mar-17


In [39]:
# remove the Month column
exchange_rates.drop(['Month'], axis=1, inplace=True)

In [44]:
exchange_rates.index = exchange_rates['period']
exchange_rates.drop(['period'], axis=1, inplace=True)

In [45]:
exchange_rates.head()

,Year,Usd rate
period,,
Jan-17,2017-01-28,305.20
Feb-17,2017-02-28,305.31
Mar-17,2017-03-28,306.40
Apr-17,2017-04-28,306.05
May-17,2017-05-28,305.54


We will now merge the two datasets together using the `period` as key.

In [52]:
food_prices_n_rates = foodprices.join(exchange_rates, on='period')

In [53]:
food_prices_n_rates.head()

,Agric eggs medium size,Agric eggs(medium size price of one),"Beans brown,sold loose",Beans:white black eye. sold loose,Beef Bone in,"Beef,boneless",Bread sliced 500g,Bread unsliced 500g,Broken Rice (Ofada),Catfish (obokun) fresh,...,Sweet potato,Tilapia fish (epiya) fresh,Titus:frozen,Tomato,"Vegetable oil:1 bottle,specify bottle",Wheat flour: prepacked (golden penny 2kg),Yam tuber,period,Year,Usd rate
0,512.99,47.42,353.60,305.53,1001.24,1249.48,302.87,270.25,377.41,899.58,...,129.43,792.18,884.82,247.55,495.29,626.53,210.58,Jan-17,2017-01-28,305.20
1,520.16,42.90,337.11,309.85,995.65,1270.67,299.75,264.86,392.04,885.13,...,127.23,795.27,942.23,236.62,507.36,621.39,215.55,Feb-17,2017-02-28,305.31
2,507.69,43.93,353.28,318.45,1010.29,1281.71,297.51,262.33,421.54,884.15,...,132.94,798.07,964.63,268.64,513.42,623.10,255.86,Mar-17,2017-03-28,306.40
3,518.66,46.22,357.19,324.04,1035.45,1323.10,296.63,277.69,425.53,886.65,...,132.43,800.77,935.00,285.72,525.72,627.33,250.30,Apr-17,2017-04-28,306.05
4,522.71,45.73,365.86,332.28,1123.65,1378.91,307.70,282.26,460.10,900.73,...,130.28,805.21,969.64,339.72,524.77,646.18,279.15,May-17,2017-05-28,305.54


In [54]:
food_prices_n_rates.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 46 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Agric eggs medium size                     59 non-null     float64       
 1   Agric eggs(medium size price of one)       59 non-null     float64       
 2   Beans brown,sold loose                     59 non-null     float64       
 3   Beans:white black eye. sold loose          59 non-null     float64       
 4   Beef Bone in                               59 non-null     float64       
 5   Beef,boneless                              59 non-null     float64       
 6   Bread sliced 500g                          59 non-null     float64       
 7   Bread unsliced 500g                        59 non-null     float64       
 8   Broken Rice (Ofada)                        59 non-null     float64       
 9   Catfish (obokun) fresh 

In [57]:
# Export the dataset as csv
food_prices_n_rates.to_csv('./datasets/foodprices_clean.csv', index=False)